In [ ]:
### 오라클 라이브러리
import cx_Oracle

In [2]:
### DB 서버 연결
dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [3]:
conn = cx_Oracle.connect("busan", "dbdb", dsn)
conn

<cx_Oracle.Connection to busan@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [4]:
cursor = conn.cursor()
cursor

<cx_Oracle.Cursor on <cx_Oracle.Connection to busan@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>>

In [5]:
### 프로시저가 반환하는 타입으로 받아올 변수 정의하기
# var(cx_Oracle.CUSROR): 오라클 CURSOR 타입의 행/열을 담 객체로
ref_cursor = cursor.var(cx_Oracle.CURSOR)
ref_cursor

<cx_Oracle.Var of type DB_TYPE_CURSOR with value None>

In [6]:
### 저장프로시저 호출
# callproc('호출할 프로시저이름', [전달할 매개변수값, 받아올 변수명])
cursor.callproc('sp_getMemberView', ['a001', ref_cursor])

['a001',
 <cx_Oracle.Cursor on <cx_Oracle.Connection to busan@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>>]

In [ ]:
### 받아온 ref_cursor 내에 객체 추출하기
# - 객체: 프로시저가 넘겨준 Sys_RefCursor 메모리
# - 데이터가 담겨있는 실제 메모리 주소 받아오기
result_cursor = ref_cursor.getvalue()
result_cursor

<cx_Oracle.Cursor on <cx_Oracle.Connection to busan@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>>

In [8]:
rows = result_cursor.fetchall()
rows

[('a001', '김은대', '대전시 동구 용운동', '대전')]

In [10]:
columns = result_cursor.description
columns

[('MEM_ID', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 15, 15, None, None, 0),
 ('MEM_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 20, 20, None, None, 0),
 ('MEM_ADD1', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 100, 100, None, None, 0),
 ('AREA', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 4000, 4000, None, None, 1)]

In [11]:
columns_nm = [data[0] for data in columns]
columns_nm

['MEM_ID', 'MEM_NAME', 'MEM_ADD1', 'AREA']

In [12]:
import pandas as pd

In [15]:
df = pd.DataFrame(rows, columns=columns_nm)
df

,MEM_ID,MEM_NAME,MEM_ADD1,AREA
0,a001,김은대,대전시 동구 용운동,대전


In [ ]:
"""
- cx_Oracle: 오라클 기능만을 컨트롤하는 라이브러리

- 판다스와 DB(ORACLE 이외 모든 DATABASE 포함) 기능을 연동해서 컨트롤해주는 라이브러리
 -> sqlalchemy 라이브러리 사용
 -> 데이터베이스의 쿼리 결과를 데이터프레임으로 직접 받아 올 수 있음
 -> 데이터프레임의 행/열 데이터를 DB에 테이블로 생성 및 저장 시키는 기능이 있음
"""